## Fantasy AI Assistant

This notebook will get you on your way to a fantasy football championship. Start by importing the Agent class we're going to use, setting the config list, and giving some system instructions. Note that `gpt-4o-mini` seems to work well. This model will be using a lot of tokens, running `gpt-4-turbo` with this notebook will ring up a large bill quickly, even with the token compression implemented below.

In [45]:
from autogen import ConversableAgent
import os 

config_list = [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]

assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant that is an expert in tool use and in fantasy football. It is long past October of 2023, you need to check the current date to search for current news."
    "Before you recommend adding or dropping any player, be very thorough and check all tools available to you to get the most up to date information possible regarding that player."
    "Return 'TERMINATE' when the task is done.",
    llm_config={"config_list": config_list},
)

Creating the user proxy agent that acts as the stand in for you.

In [46]:
# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)


Now we start to create the tools that our assistant can use. There is potential for some change here, as these are written as tools rather than functions. When `autogen` hits their roadmap target of letting docker code execution agents take in user-defined functions, I will refactor this.

In [47]:
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool
from autogen import register_function

def google_it(query: str) -> str:
    "Search Google for keywords"
    #this is going to assume you have your environment variables set for GOOGLE_API and GOOGLE_CSE
    search = GoogleSearchAPIWrapper()

    google_search = Tool(
        name="google_search",
        description="Search Google for recent results.",
        func=search.run)
    
    return google_search({'query': query})

register_function(
    google_it,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="google_it",  # By default, the function name is used as the tool name.
    description="Searches google for recent results using the input as keywords",  # A description of the tool.
)


In [48]:
from langchain_community.tools.reddit_search.tool import RedditSearchRun
from langchain_community.utilities.reddit_search import RedditSearchAPIWrapper
from langchain_community.tools.reddit_search.tool import RedditSearchSchema

reddit_client_id = os.environ['REDDIT_CLIENT_ID']
reddit_client_secret = os.environ['REDDIT_CLIENT_SECRET']
reddit_user_agent = "fantasy-assistant"

def check_reddit(player: str, player_team_subreddit: str) -> str:
    "Check Reddit for any recent player news. Inputs are the player and the subreddit for that player's team"

    r_search = RedditSearchRun(
        api_wrapper=RedditSearchAPIWrapper(
            reddit_client_id=reddit_client_id,
            reddit_client_secret=reddit_client_secret,
            reddit_user_agent=reddit_user_agent,
        )
    )

    search_params = RedditSearchSchema(
        query=player, sort="new", time_filter="week", subreddit="fantasyfootball", limit="5"
        )

    result = r_search.run(tool_input=search_params.dict())

    try:
        search_params = RedditSearchSchema(
            query=player, sort="new", time_filter="week", subreddit=player_team_subreddit, limit="5"
            )
        extra_result = r_search.run(tool_input=search_params.dict())
        result += extra_result
    except:
        pass
    return result

register_function(
    check_reddit,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="check_reddit",  # By default, the function name is used as the tool name.
    description="Check Reddit for any recent player news. Inputs are the player and the subreddit for that player's team",  # A description of the tool.
)

Here you need to make sure you've grabbed your `swid` and `s2` from ESPN's website. It's fairly straightforward when you go to developer tools after logging in. You also need to set up a `config.py` per the comment below the import. It needs to have a variable called `league_team_info` that contains a list of tuples. The values in the tuple must be your nickname for the league, the league ID, and your team's ID within the league. 

In [49]:
import os
from espn_api.football import League
from config import league_team_info
#[('work-friends',1234567,7)]
from espn_ff_toolkit import get_espn_leagues 


swid = os.environ['ESPN_SWID']
espn_s2 = os.environ['ESPN_S2']

year=2024
league_dict = get_espn_leagues(league_team_info,year,espn_s2=espn_s2,swid=swid)

In [50]:
from espn_ff_toolkit import get_rosters, get_free_agents, get_roster_and_projections

def get_rosters_tool()-> str:
    "Get the rosters for all current leagues"
    return get_rosters(league_dict)

register_function(
    get_rosters_tool,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description = "Get the rosters for all current leagues"
)

def get_free_agents_tool() -> str:
    "Get the current free agents in all leagues"
    return get_free_agents(league_dict)

register_function(
    get_free_agents_tool,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description = "Get the current free agents in all leagues"
)

def get_roster_and_projections_tool() -> str:
    "Get the rosters for all current leagues and also all projected and past stats"
    return get_roster_and_projections(league_dict)

register_function(
    get_roster_and_projections_tool,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description = "Get the rosters for all current leagues and also all projected and past stats"
)

In [51]:
from datetime import datetime
def get_current_date() -> str:
    "Get the current date, because it is not October 2023"
    current_date = datetime.now()
    return str(current_date)

register_function(
    get_current_date,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description="Find out what the current date is",  # A description of the tool.
)

In [52]:
from draft_kings import Sport, Client

def get_draftkings_weekly_salaries():
    output = {}
    players_info = Client().available_players(draft_group_id=109136)
    for p in players_info.players:
        full_name = p.first_name + " " + p.last_name
        output[full_name] = p.draft_details.salary
    return output

dk_salaries = get_draftkings_weekly_salaries()

def check_draftkings_salary(player: str) -> str:
    "Check a player's salary on DraftKings. A higher salary means they are projected to do better"
    try:
        salary = dk_salaries[player]
        return(player, "salary on DraftKings is: ", salary)
    except:
        return(player, "has no salary on DraftKings this week")

register_function(
    check_draftkings_salary,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description="Check a player's salary on DraftKings. A higher salary means they are projected to do better",  # A description of the tool.
)


In [53]:
from autogen.agentchat.contrib.capabilities.text_compressors import LLMLingua
from autogen.agentchat.contrib.capabilities.transforms import TextMessageCompressor
from autogen.agentchat.contrib.capabilities import transform_messages

llm_lingua = LLMLingua()
text_compressor = TextMessageCompressor(text_compressor=llm_lingua)
context_handling = transform_messages.TransformMessages(transforms=[text_compressor])
context_handling.add_to_agent(assistant)

In [ ]:
chat_result = user_proxy.initiate_chat(assistant, message="Check all my rosters, then give me any recent news about those players and tell me their salaries on DraftKings, if they have one")

In [55]:
from IPython.display import Markdown as md
md(chat_result.summary)

Here's the current information on players from both leagues including their projected stats and DraftKings salaries:

### League: Squad
- **Justin Jefferson**
  - Projected Points: 229.22
  - DraftKings Salary: $8400

- **Isiah Pacheco**
  - Projected Points: 209.83
  - DraftKings Salary: Not available

- **Mike Evans**
  - Projected Points: 196.80
  - DraftKings Salary: $7300

- **Brandon Aiyuk**
  - Projected Points: 184.85
  - DraftKings Salary: Not available

- **Lamar Jackson**
  - Projected Points: 303.98
  - DraftKings Salary: Not available

- **Mark Andrews**
  - Projected Points: 153.14
  - DraftKings Salary: Not available

- **Rhamondre Stevenson**
  - Projected Points: 168.88
  - DraftKings Salary: $5900

- **Terry McLaurin**
  - Projected Points: 157.06
  - DraftKings Salary: $5600

- **Brian Robinson Jr.**
  - Projected Points: 160.98
  - DraftKings Salary: $5200

- **Ladd McConkey**
  - Projected Points: 142.37
  - DraftKings Salary: $4700

- **DeAndre Hopkins**
  - Projected Points: 145.01
  - DraftKings Salary: $5700

- **Brock Bowers**
  - Projected Points: 115.72
  - DraftKings Salary: $4500

- **Trey Benson**
  - Projected Points: 110.83
  - DraftKings Salary: $5100

- **Jake Elliott**
  - DraftKings Salary: Not available

- **Trevor Lawrence**
  - Projected Points: 249.08
  - DraftKings Salary: $6200

- **Keaton Mitchell**
  - DraftKings Salary: Not available

- **Bucky Irving**
  - Projected Points: 98.81
  - DraftKings Salary: $4800

### League: Work
- **Christian McCaffrey**
  - Projected Points: 301.83
  - DraftKings Salary: Not available

- **Marvin Harrison Jr.**
  - Projected Points: 186.73
  - DraftKings Salary: $7200

- **Alvin Kamara**
  - Projected Points: 197.18
  - DraftKings Salary: $6700

- **Jaylen Waddle**
  - Projected Points: 178.81
  - DraftKings Salary: $6300

- **Trey McBride**
  - Projected Points: 140.96
  - DraftKings Salary: $6100

- **Joe Burrow**
  - Projected Points: 278.75
  - DraftKings Salary: $6500

- **Christian Kirk**
  - Projected Points: 163.68
  - DraftKings Salary: $5500

- **Nick Chubb**
  - Projected Points: 125.75
  - DraftKings Salary: $5400

- **Jaleel McLaughlin**
  - Projected Points: 117.83
  - DraftKings Salary: $4300

- **Joshua Palmer**
  - Projected Points: 129.51
  - DraftKings Salary: $5200

- **Gabe Davis**
  - Projected Points: 144.51
  - DraftKings Salary: $4500

- **Harrison Butker**
  - DraftKings Salary: Not available

- **Rashid Shaheed**
  - Projected Points: 131.58
  - DraftKings Salary: $4400

- **MarShawn Lloyd**
  - DraftKings Salary: Not available

- **Kimani Vidal**
  - Projected Points: 33.27
  - DraftKings Salary: $4700

This concludes the checks for roster players, recent news, and their salaries on DraftKings. If you need further assistance or specific player discussions, feel free to ask! 

